In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_dir = '../input/stanford-car-dataset-by-classes-folder/car_data/car_data/train/'
test_dir = '../input/stanford-car-dataset-by-classes-folder/car_data/car_data/test/'
img_height = 224
img_width = 224
batch_size=16

In [ ]:
train_datagen = ImageDataGenerator(
    shear_range=10,
    zoom_range=0.2,
    horizontal_flip=True,
    preprocessing_function=preprocess_input
    ,validation_split=0.3)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    batch_size=batch_size,
    class_mode='categorical',
    target_size=(img_height,img_width))

validation_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input)

validation_generator = train_datagen.flow_from_directory(
    test_dir,
    shuffle=False,
    class_mode='categorical',
    target_size=(img_height,img_width))

In [ ]:
model = ResNet50(
    weights = 'imagenet',
    include_top = False, 
    input_shape = [img_height, img_width, 3]
    )

for layer in model.layers:
    layer.trainable = False

pool_layer = layers.GlobalAveragePooling2D()(model.output)
# kernel_regularizer=tf.keras.regularizers.l2(0.01)
output_layer = layers.Dense(196)(pool_layer)
model = Model(model.inputs, output_layer)
opt = SGD(learning_rate=0.01, momentum=0.9)
model.compile(optimizer=opt,loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
hist = model.fit(train_generator,validation_data=validation_generator, epochs=40, verbose=1)

In [ ]:
hist.history.keys()

In [ ]:
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()